In [1]:
# Import important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from functools import reduce

from collections import Counter

import warnings 
warnings.simplefilter("ignore")

## Introduction
In the cell below, one can enter a district number in de Mekong Delta. Then, when running the file, an excel file is created for that district, with all input data for the ABM. 
This file uses data of Pop Housing Census 2009 & 2019, and VHLSS2014.

In [2]:
# Decide on a district
district = 894

## Pop housing census 2009
From this dataset, the following is used:
- Age distribution
- Household size
- Difficulties
- Is the person working yes/no


It should be mentioned that the weight is per household member. Therefore, when saying things about the household as a whole, weight cannot be used. 

In [3]:
# Define path
path = os.getcwd()

# Import data
path_sav = path + "\Census2009_MDMigration_Q1Q34.sav"
df=pd.read_spss(path_sav)
df_mekong = df

In [4]:
# Only select provinces of the Mekong Delta, instead of Vietnam as a whole
df_mekong = df[(df['Tinh'] >= 82) & ((df['Tinh']) <= 96)]
# Rename district column
df_mekong = df_mekong[df_mekong['Huyen']==district]
# Create a household id to group household members together
df_mekong["Household_id"] = df_mekong["Tinh"].astype(str) + " - " + df_mekong["Huyen"].astype(str) + " - " + df_mekong["Xa"].astype(str) + " - " + df_mekong["Dban"].astype(str) + " - " + df_mekong["Hoso"].astype(str)
# Select rural area
df_mekong = df_mekong[df_mekong['TTNT'] == 2]
# Select the correct district
df_mekong = df_mekong[df_mekong['Huyen'] == district]


### Age distribution

In [5]:
df_ages = df_mekong

# Create a function to transform the ages in groups, to see how many people are in each group.
def age_category(age):
    if age <= 15:
        return "0-15"
    elif age <= 45:
        return "16-45"
    elif age <= 59:
        return "46-59"
    else:
        return "59-85"

# Apply the function, and name the new column "age group"
df_ages['age_group'] = df_ages['C5'].apply(age_category)

# Count per household the number of in each age group
age_counts = df_ages.groupby(['Household_id', 'age_group']).size().unstack(fill_value=0)
# Look at the average number of people per household
average_distribution = age_counts.mean()
# Translate this to percentages
age_distribution_percentages = (average_distribution / average_distribution.sum()).reset_index()
age_distribution_percentages.columns = ['age_group', 'percentage']
# Save this in the Excel file
with pd.ExcelWriter(f"model_input_data_{district}.xlsx") as writer:
    age_distribution_percentages.to_excel(writer, sheet_name="age_distribution", index=False)

### Household size

In [6]:
# Groupby household_id 
household_sizes = df_mekong.groupby('Household_id').size()

# Look at the mean and std dev
mean = household_sizes.mean()
std_devs = household_sizes.std()

# Put the mean and std dev in a dataframe
hh_size_stats = pd.DataFrame({'hh_size': [mean, std_devs]}, index=['mean', 'std_dev'])
hh_size_stats

# Save it in Excel
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    hh_size_stats.to_excel(writer, sheet_name="Household_size", index=True)

### Difficulties for household members

In [7]:
# Select the correct columns
difficulties_df = df_mekong[['Household_id', "C11A", "C11B", "C11C", "C11D", "C5"]]

# This is on individual level, so the weights can be used to make it more representable
difficulties_df = difficulties_df.loc[difficulties_df.index.repeat(df_mekong['WEIGHT'])].reset_index(drop=True)

# The level of difficulty is in Vietnamese, and needs to be translated
transformation_difficulties = {
    "Kh«ng khã kh¨n": "No_difficulty",
    "Khã kh¨n": "Some_difficulty",
    "RÊt khã kh¨n": "Very_difficulty",
    "Kh«ng thÓ": "Unable"}

difficulties_df['C11A'] = difficulties_df['C11A'].replace(transformation_difficulties)
difficulties_df['C11B'] = difficulties_df['C11B'].replace(transformation_difficulties)
difficulties_df['C11C'] = difficulties_df['C11C'].replace(transformation_difficulties)
difficulties_df['C11D'] = difficulties_df['C11D'].replace(transformation_difficulties)

# Also use the age group function from above, so it is possible to see the level of difficulty per age group
difficulties_df['age_group'] = difficulties_df['C5'].apply(age_category)

# Rename columns
difficulties_df = difficulties_df.rename(columns = {"C11A":"Seeing","C11B": "Hearing", "C11C":"Walking", "C11D":"Remembering"})

# Check distributions
functions = ['Hearing', 'Seeing', 'Walking', 'Remembering']
results = {}

for function in functions:
    # Calculate per age group and function the level of difficulty
    distribution = (
        difficulties_df.groupby('age_group')[function]
          .value_counts(normalize=True)  
          .unstack(fill_value=0)         
          * 100                                               
    )
    results[function] = distribution

# Set results in a dataframe
df_result_difficulties = pd.concat(results, names=['function'])
df_result_difficulties = df_result_difficulties.drop([9.0], axis=1) # There is a random 9, delete this one

# Save it to excel
with pd.ExcelWriter(f"model_input_data_{district}.xlsx" ,mode='a', engine='openpyxl') as writer:
    df_result_difficulties.to_excel(writer, sheet_name="Dissabilities", index=True)

### Education level

In [8]:
df_education = df_mekong[['Household_id', 'C5', 'Educ','C12' ]]

# Divide the ages in age groups
df_education['age_group'] = df_education['C5'].apply(age_category)

education_percentages = (
    df_education.groupby('age_group')['Educ']
      .value_counts(normalize=True)
      .unstack(fill_value=0)
      * 100)

with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    education_percentages.to_excel(writer, sheet_name="Education_level", index=True)


### Working yes/no

In [9]:
# Select correct columns
working_df = df_mekong[['Household_id', 'C20', 'C5']]

# This is on individual level, so the weights can be used to make it more representable
working_df = working_df.loc[working_df.index.repeat(df_mekong['WEIGHT'])].reset_index(drop=True)

# Translate the vietnamese to numbers. 1 means the person is working, 0 means they are not working currently. 
transformation_work = {
    "1. Cã": 1,
    "2. Kh«ng": 0,
    "3. Kh«ng biÕt": 0}

working_df['C20'] = working_df['C20'].replace(transformation_work)
# Look at the ages to say something per age group
working_df['age_group'] = working_df['C5'].apply(age_category)
# Fill NaN values with zero
working_df.fillna(0, inplace=True)
working_df['C20'] = working_df['C20'].astype(int)

# Look at the total number of people per age group, and count which of them are working
hh_age = working_df.groupby(['Household_id', 'age_group'])['C20'].agg(
    working='sum', # possible to take the sum since 0 means they are not working and 1 means working
    total='count'
).reset_index()

# Not working are the total people minus the working people
hh_age['not_working'] = hh_age['total'] - hh_age['working']

# Look at the averages per age group
average_per_agegroup = hh_age.groupby('age_group')[['working', 'not_working']].mean().round(2)

# Put it in percentage
average_per_agegroup = average_per_agegroup.div(average_per_agegroup.sum(axis=1), axis=0).round(2) * 100

# Save it in excel
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    average_per_agegroup.to_excel(writer, sheet_name="Percent_working_per_age", index=True)



## VHLSS 2014
From this dataset is used: 
- Land category (small, medium, large) combined with dominant income source(e.g. annual crops)
- Average costs of living
- Are people connected to an association?

All data is in different datasets, and weights are unclear

### Land data
Since this data is not representative (the weight is not clear), I decided to focus on the Mekong Delta as a whole, and not select a province.

In [13]:
# Define path
path = os.getcwd()

# Import data
path_ = path + "\Data2014VHLSS\\"

# create empty list to store dataframes
dataframes = []

# Select correct columns
datasets_info = {
    "muc4b11.xlsx": [  # Rice
        {"column": "m4b11c3", "new_name": "Rice_land_size", "filter": False}
    ],
    "muc4b12.xlsx": [  # Annual crops
        {"column": "m4b12c3", "new_name": "Annual_crops_land_size", "filter": False}
    ],
    "muc4b13.xlsx": [  # Industrial crops
        {
            "column": "m4b13c3a",
            "check_column": "m4b13c3b",
            "new_name": "Industrial_crops_land_size",
            "filter": True
        }
    ],
    "muc4b14.xlsx": [  # Fruit trees
        {
            "column": "m4b14c3a",
            "check_column": "m4b14c3b",
            "new_name": "Fruit_trees_land_size",
            "filter": True
        }
    ]
}

def process_dataset(file_path, col_info):
    cols_to_read = ["diaban", "huyen", "tinh", "xa", "hoso", col_info["column"]]
    
    if col_info.get("filter", False):
        cols_to_read.append(col_info["check_column"])
    
    df = pd.read_excel(file_path, usecols=cols_to_read)
    
    # For fruit trees and industrial crops, the data is in M2 and number of seeds. Number of seeds depends on the crop (e.g. coconut has a different number of seeds than bananas), therefore I decided to only focus on M2 data
    if col_info.get("filter", False):
        df = df[df[col_info["check_column"]] == "M2"]
    
    # Only select provinces of the Mekong Delta, instead of all provinces in Vietnam
    mekong_delta_provinces = ['TØnh Long An','TØnh TiÒn Giang', 'TØnh BÕn Tre' , 'TØnh Trµ Vinh', 'TØnh VÜnh Long', 'TØnh §ång Th¸p', 'TØnh An Giang', 'TØnh Kiªn Giang', 'TØnh CÇn Th¬', 'TØnh HËu Giang', 'TØnh Sãc Tr¨ng', 'TØnh B¹c Liªu', 'TØnh Cµ Mau',
                              80,81,82,83,84,85,86,87,88,89,80,81,92,93,94,95,96]
    df = df[df['tinh'].isin(mekong_delta_provinces)]

    provinces = {'TØnh Long An':"80",'TØnh TiÒn Giang':"82", 'TØnh BÕn Tre':"83" , 'TØnh Trµ Vinh':"84", 'TØnh VÜnh Long':"86", 
                 'TØnh §ång Th¸p':"87", 'TØnh An Giang':"89", 'TØnh Kiªn Giang':"91", 'TØnh CÇn Th¬':"92", 'TØnh HËu Giang':"93", 
                 'TØnh Sãc Tr¨ng':"94", 'TØnh B¹c Liªu':"95", 'TØnh Cµ Mau':"96"}

    # In not all datasets the province is mentioned the same. Therefore, we need to convert this
    def convert_province(val):
        if isinstance(val, str):
            return provinces.get(val.strip(), None)
        return val 
    df['tinh'] = df['tinh'].apply(convert_province)

    # Rename columns
    df = df.rename(columns={
        "tinh": "Province",
        "huyen": "District",
        "xa": "Commune",
        "hoso": "Household",
        col_info["column"]: col_info["new_name"]
    })
    
    df["Household_id"] = df["Province"].astype(str) + " - " + df['District'].astype(str) + " - " + df["Commune"].astype(str) + " - " + df["diaban"].astype(str) + " - " + df["Household"].astype(str) 

    df = df.drop(["Commune", "Household", "District", "Province", "diaban"], axis=1)
    df.fillna(0, inplace=True)
    df = df.groupby("Household_id", as_index=False).mean(numeric_only=True)

    return df

dataframes_land_size = []

for file_name, col_info_list in datasets_info.items():
    file_path = f"{path_}\\{file_name}"
    
    for col_info in col_info_list:
        df = process_dataset(file_path, col_info)
        dataframes_land_size.append(df)


In [14]:
# Combine all data in 1 dataset
df_merged_land = dataframes_land_size[0]
for df in dataframes_land_size[1:]:
    df_merged_land = df_merged_land.merge(df, on=["Household_id"], how="outer")

# fill in nan values with 0
df_merged_land = df_merged_land.fillna(0, inplace=False)

# Get an overview of the land sizes and total land size
df_merged_land['Total_land_size'] = df_merged_land['Rice_land_size'] + df_merged_land['Annual_crops_land_size']+ df_merged_land['Industrial_crops_land_size']+ df_merged_land['Fruit_trees_land_size']

### Get employment data

In [15]:
# Define path
path = os.getcwd()

path_ = path + "\Data2014VHLSS\\muc4a.xlsx"
employ_data=pd.read_excel(path_, usecols=('diaban', 'tinh', 'xa', 'huyen','hoso','m4ama', 'm4ac1a', 'm4ac1b', 'm4ac1c'))

mekong_delta_provinces = ['TØnh Long An','TØnh TiÒn Giang', 'TØnh BÕn Tre' , 'TØnh Trµ Vinh', 'TØnh VÜnh Long', 'TØnh §ång Th¸p', 'TØnh An Giang', 'TØnh Kiªn Giang', 'TØnh CÇn Th¬', 'TØnh HËu Giang', 'TØnh Sãc Tr¨ng', 'TØnh B¹c Liªu', 'TØnh Cµ Mau',
80,81,82,83,84,85,86,87,88,89,80,81,92,93,94,95,96]
employ_data = employ_data[employ_data['tinh'].isin(mekong_delta_provinces)]

provinces = {'TØnh Long An':"80",'TØnh TiÒn Giang':"82", 'TØnh BÕn Tre':"83" , 'TØnh Trµ Vinh':"84", 'TØnh VÜnh Long':"86", 'TØnh §ång Th¸p':"87", 'TØnh An Giang':"89", 'TØnh Kiªn Giang':"91", 'TØnh CÇn Th¬':"92", 'TØnh HËu Giang':"93", 'TØnh Sãc Tr¨ng':"94", 'TØnh B¹c Liªu':"95", 'TØnh Cµ Mau':96}

def convert_province(val):
    if isinstance(val, str):
        return provinces.get(val.strip(), None)
    return val 
employ_data['tinh'] = employ_data['tinh'].apply(convert_province)

# Rename columns
employ_data = employ_data.rename(columns = {'tinh':'Province', 'xa':'Commune', 'huyen':"District", 'hoso':'Household','m4ama':'Member_id', 'm4ac1a':'Worked_wage', 'm4ac1b':'Worked_self_agri',
 'm4ac1c':'Worked_self_nonAgri' })

# Create new column to make it easier to match with other datasets
employ_data["Household_id"] = employ_data["Province"].astype(str) + " - " + employ_data['District'].astype(str)+ " - " + employ_data["Commune"].astype(str) + " - " + employ_data["diaban"].astype(str) + " - " + employ_data["Household"].astype(str)


# since in the data 1=yes, 2=no, change 2 to 0 to have a dummy variable
employ_data["Worked_wage"] = employ_data['Worked_wage'].replace("Yes", 1)
employ_data["Worked_wage"] = employ_data['Worked_wage'].replace("No", 0)
employ_data["Worked_self_agri"] = employ_data['Worked_self_agri'].replace("Yes", 1)
employ_data["Worked_self_agri"] = employ_data['Worked_self_agri'].replace("No", 0)
employ_data["Worked_self_nonAgri"] = employ_data['Worked_self_nonAgri'].replace("Yes", 1)
employ_data["Worked_self_nonAgri"] = employ_data['Worked_self_nonAgri'].replace("No", 0)

# Select relevant columns
employ_data = employ_data.drop(['diaban', 'Commune', 'Household', 'District', 'Province', 'Member_id'], axis=1)

# Group on household level instead of individual level within household
employ_data = employ_data.groupby("Household_id", as_index=False).sum()

### Get income data to see the dominant income source

In [16]:
# Define path
path = os.getcwd()

# Import data
path_ = path + "\Data2014VHLSS\\"

# create empty list to store dataframes
dataframes = []

# Select relevant columns
datasets_info = {
    "Ho1.xlsx": [
        {"column": "m2vtn", "new_name": "Subsidies"},
        {"column": "m2xtn", "new_name": "Scholarships"},
        {"column": "m2xct", "new_name": "Cost_education_1"},
        {"column": "m2vct", "new_name": "Cost_education_2"},
        {"column": "m2dct", "new_name": "Cost_education_3"}
    ],

    "Ho2.xlsx": [
        {"column": "m4atn", "new_name": "Income_salary_wages/pensions"}
    ],

    "muc4b11.xlsx": [
        {"column": "m4b11c4", "new_name": "Rice_harvest"},
        {"column": "m4b11c8", "new_name": "Rice_revenue"}],

    "muc4b12.xlsx": [
        {"column": "m4b12c4", "new_name": "Annual_crops_harvest"},
        {"column": "m4b12c7", "new_name": "Annual_crops_revenue"}],

    "muc4b13.xlsx": [{"column": "m4b13c4", "new_name": "Industrial_crops_harvest"}, 
        {"column": "m4b13c7", "new_name": "Industrial_crops_revenue"}],

    "muc4b14.xlsx": [{"column": "m4b14c4", "new_name": "Fruit_trees_harvest"},
        {"column": "m4b14c7", "new_name": "Fruit_trees_revenue"}],

    "muc4b15.xlsx": [{"column": "m4b15c5", "new_name": "Revenue_by_products"}],

    "muc4b16.xlsx": [{"column": "m4b16c2a", "new_name": "Cost_rice"},
        {"column": "m4b16c2b", "new_name": "Cost_annual_crops"},
        {"column": "m4b16c2c", "new_name": "Cost_industrial_crops"},
        {"column": "m4b16c2d", "new_name": "Cost_fruit_trees"}],
    "muc4b21.xlsx": [{"column": "m4b21c5", "new_name": "Revenue_hunting_etc"}],
    "muc4b22.xlsx": [{"column": "m4b22c19", "new_name": "Costs_hunting_etc"}],
    "muc4b31.xlsx": [{"column": "m4b31c5", "new_name": "Revenue_machines"}],
    "muc4b32.xlsx": [{"column": "m4b32c17", "new_name": "Costs_machines"}],
    "muc4b41.xlsx": [{"column": "m4b41c3f", "new_name": "Revenues_foresty"}],
    "muc4b42a.xlsx": [{"column": "m4b42c14", "new_name": "Costs_foresty"}],
    "muc4b51.xlsx": [{"column": "m4b51c5", "new_name": "Revenues_aqua"}],
    "muc4b52.xlsx": [{"column": "m4b52c19", "new_name": "Costs_aqua"}],
    "muc4c1.xlsx": [{"column": "m4c1c18", "new_name": "Revenue_processing_aqua_agri"}],
    "muc4c2.xlsx": [{"column": "m4c2c33", "new_name": "Costs_processing_aqua_agri"}],
    "muc4d.xlsx": [{"column": "m4dc2", "new_name": "Other_revenues"}]
}


In [17]:
# # Open all dataframes and columns as specified above
for file_name, col_info_list in datasets_info.items():
    file_path = f"{path_}\\{file_name}"

    for col_info in col_info_list:
        # Select the correct columns
        df = pd.read_excel(file_path, usecols=["diaban", "huyen", "tinh", "xa", "hoso", col_info["column"]])
        
        # Only select the Mekong Delta
        mekong_delta_provinces = ['TØnh Long An','TØnh TiÒn Giang', 'TØnh BÕn Tre' , 'TØnh Trµ Vinh', 'TØnh VÜnh Long', 'TØnh §ång Th¸p', 'TØnh An Giang', 'TØnh Kiªn Giang', 'TØnh CÇn Th¬', 'TØnh HËu Giang', 'TØnh Sãc Tr¨ng', 'TØnh B¹c Liªu', 'TØnh Cµ Mau',
        80,81,82,83,84,85,86,87,88,89,80,81,92,93,94,95,96]
        df = df[df['tinh'].isin(mekong_delta_provinces)]

        provinces = {'TØnh Long An':"80",'TØnh TiÒn Giang':"82", 'TØnh BÕn Tre':"83" , 'TØnh Trµ Vinh':"84", 'TØnh VÜnh Long':"86", 
        'TØnh §ång Th¸p':"87", 'TØnh An Giang':"89", 'TØnh Kiªn Giang':"91", 'TØnh CÇn Th¬':"92", 'TØnh HËu Giang':"93", 'TØnh Sãc Tr¨ng':"94", 'TØnh B¹c Liªu':"95", 'TØnh Cµ Mau':96}

        # Not every dataset uses the same province name, so we need to convert that
        def convert_province(val):
            if isinstance(val, str):
                return provinces.get(val.strip(), None)
            return val 
        df['tinh'] = df['tinh'].apply(convert_province)
        
        # Rename columns
        df = df.rename(columns={
            "tinh": "Province",
            "huyen":"District",
            "xa": "Commune",
            "hoso": "Household",
            col_info["column"]: col_info["new_name"]
        })

        #Create Household id
        df["Household_id"] = df["Province"].astype(str) + " - " + df['District'].astype(str) + " - " + df["Commune"].astype(str) + " - " + df["diaban"].astype(str) + " - " + df["Household"].astype(str) 
        
        
        # Drop irrelevant columns
        df = df.drop(["Commune", "Household", "District","Province", "diaban"], axis=1)
        
        # Change Nan values to zero
        df.fillna(0, inplace=True)
        
        # Group by household ID to get the total income per household
        df = df.groupby("Household_id", as_index=False).sum()
        
        
        # Collect all dataframes 
        dataframes.append(df)

In [18]:
# Merge dataframes together into 1 dataset
df_merged = dataframes[0]
for df in dataframes[1:]:
    df_merged = df_merged.merge(df, on=["Household_id"], how="outer")

# fill in nan values with 0
df_merged = df_merged.fillna(0, inplace=False)

income_df = df_merged

# Create new columns
income_df['Cost_education'] = income_df['Cost_education_1'] + income_df['Cost_education_2'] + income_df['Cost_education_3']
income_df['Subsidy_education'] = income_df['Scholarships'] + income_df['Subsidies']
income_df = income_df.drop(['Cost_education_1', 'Cost_education_2', 'Cost_education_3',"Scholarships", "Subsidies" ], axis=1)

income_df['Total_hh_income'] = income_df['Subsidy_education'] - income_df['Cost_education'] + income_df['Income_salary_wages/pensions'] + income_df['Rice_revenue']
+ income_df['Annual_crops_revenue']+ income_df['Industrial_crops_revenue']+ income_df['Fruit_trees_revenue'] + income_df['Revenue_hunting_etc']
+ income_df['Revenue_machines'] + income_df['Revenues_foresty']+ income_df['Revenues_aqua'] + income_df['Revenue_processing_aqua_agri']+ income_df['Other_revenues']
+ income_df['Revenue_by_products'] - income_df['Cost_rice']- income_df['Cost_annual_crops']- income_df['Cost_industrial_crops']- income_df['Cost_fruit_trees']
- income_df['Costs_hunting_etc']- income_df['Costs_machines']- income_df['Costs_foresty']- income_df['Costs_aqua']- income_df['Costs_processing_aqua_agri']



0            -0.0
1            -0.0
2            -0.0
3            -0.0
4            -0.0
           ...   
13911   -137000.0
13912   -147500.0
13913   -158650.0
13914    -28300.0
13915    -38300.0
Length: 13916, dtype: float64

#### Merge income data with land data

In [19]:
df_land_income = df_merged_land.merge(income_df, on=["Household_id"], how="right")

# fill in nan values with 0
df_land_income = df_land_income.fillna(0, inplace=False)

# delete all rows who do not have income from salary/wage or revenues
income_cols = ['Rice_revenue', 'Annual_crops_revenue', 'Industrial_crops_revenue', 'Fruit_trees_revenue', 'Revenue_by_products', 'Revenue_hunting_etc', 'Revenue_machines', 'Revenues_foresty', 'Revenues_aqua', 'Revenue_processing_aqua_agri', 'Income_salary_wages/pensions']
df_filtered = df_land_income[(df_land_income[income_cols] <= 0).all(axis=1)]

df_land_income = df_land_income.drop(df_filtered.index)

# Select households which have land
land_households = df_land_income[df_land_income['Total_land_size'] > 0]
# Give the households a dominant income source, based on their income
land_households['Dominant_income_source'] = land_households[income_cols].idxmax(axis=1)

### Match land and income data and employment data

In [20]:
# Merge land and income data with the employment data
land_employ_data = land_households.merge(employ_data, on= 'Household_id', how = "inner")

# Divide the lands in 3 bins: small, medium, large
land_employ_data['land_size_type'] = pd.cut(land_employ_data['Total_land_size'], bins=[0, 5000, 20000, float('inf')],labels=['small', 'medium', 'large'],right=False)

# Check how many times each land category is there
land_sizes = land_employ_data['land_size_type'].value_counts(normalize=True).reset_index()
land_sizes.columns = ['land_size_type', 'percentage']
land_sizes

# Upload to the excel file
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    land_sizes.to_excel(writer, sheet_name="Land_sizes", index=False)

In [21]:
land_employ_data

,Household_id,Rice_land_size,Annual_crops_land_size,Industrial_crops_land_size,Fruit_trees_land_size,Total_land_size,Income_salary_wages/pensions,Rice_harvest,Rice_revenue,Annual_crops_harvest,...,Costs_processing_aqua_agri,Other_revenues,Cost_education,Subsidy_education,Total_hh_income,Dominant_income_source,Worked_wage,Worked_self_agri,Worked_self_nonAgri,land_size_type
0,80 - 794 - 27685 - 1 - 10,11250.0,0.000000,0.0,0.0,11250.000000,23000.0,24150.0,140970.0,0.0,...,0.0,500.0,6480.0,0.0,157490.0,Rice_revenue,1.0,1.0,0.0,medium
1,80 - 794 - 27685 - 1 - 15,1500.0,0.000000,0.0,0.0,1500.000000,5300.0,3140.0,18916.0,0.0,...,0.0,47000.0,0.0,0.0,24216.0,Rice_revenue,1.0,1.0,0.0,small
2,80 - 794 - 27685 - 1 - 2,1500.0,0.000000,0.0,0.0,1500.000000,71340.0,3220.0,18788.0,0.0,...,0.0,200.0,1920.0,140.0,88348.0,Income_salary_wages/pensions,2.0,1.0,0.0,small
3,80 - 794 - 27685 - 1 - 4,3750.0,0.000000,0.0,0.0,3750.000000,46200.0,7800.0,46600.0,0.0,...,274600.0,5000.0,23515.0,140.0,69425.0,Revenue_processing_aqua_agri,1.0,1.0,1.0,small
4,80 - 794 - 27692 - 5 - 10,4500.0,0.000000,0.0,0.0,4500.000000,5600.0,6700.0,39312.0,0.0,...,730.0,600.0,23555.0,0.0,21357.0,Revenue_machines,1.0,1.0,1.0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,96 - 973 - 32242 - 18 - 3,0.0,27.500000,0.0,0.0,27.500000,11040.0,0.0,0.0,160.0,...,0.0,1000.0,0.0,0.0,11040.0,Revenues_aqua,1.0,3.0,0.0,small
4117,96 - 973 - 32248 - 2 - 13,0.0,40.000000,0.0,0.0,40.000000,12000.0,0.0,0.0,160.0,...,0.0,0.0,20000.0,0.0,-8000.0,Revenues_aqua,1.0,3.0,0.0,small
4118,96 - 973 - 32248 - 2 - 6,0.0,21.666667,0.0,0.0,21.666667,0.0,0.0,0.0,110.0,...,0.0,0.0,45200.0,0.0,-45200.0,Revenues_aqua,0.0,2.0,0.0,small
4119,96 - 973 - 32248 - 2 - 7,0.0,35.000000,0.0,0.0,35.000000,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,Revenues_aqua,0.0,1.0,0.0,small


### Work type per land size

In [22]:
# Group data by land size type and dominant income source, while keeping the number of wage workers, worked self agri and worked self nonagri into account
grouped_income_landsize2 = land_employ_data.groupby(['land_size_type', 'Dominant_income_source'])[['Worked_wage', 'Worked_self_agri', 'Worked_self_nonAgri']].mean()

# I decided to change the scope of this project. Industrial crops and fruit trees are referred to as perennial crops (same as the pop housing census names). 
# Rice is referred to as "Rice"
# I decided to not take foresty etc into account. This may be an option for the future
grouped_income_landsize2 = grouped_income_landsize2.reset_index()
combine_map = {
    'Industrial_crops_revenue': 'Perennial crops',
    'Fruit_trees_revenue': 'Perennial crops',
    'Rice_revenue': 'Rice',
    'Annual_crops_revenue':'Annual crops',
    'Income_salary_wages/pensions': "wages",
    'Revenues_aqua': 'Aquaculture'
}
grouped_income_landsize2['Dominant_income_source'] = grouped_income_landsize2['Dominant_income_source'].replace(combine_map)
grouped_income_landsize2 = grouped_income_landsize2.groupby(['Dominant_income_source', 'land_size_type'], as_index=False).mean()

# Select relevant columns
grouped_income_landsize2 = grouped_income_landsize2[['Dominant_income_source', 'land_size_type', 'Worked_wage', 'Worked_self_agri', 'Worked_self_nonAgri']]
grouped_income_landsize2

# Save to the excel file
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    grouped_income_landsize2.to_excel(writer, sheet_name="Work_type_per_land_size", index=False)

### Average living costs


In [23]:
path = os.getcwd()

# Import data
path_ = path + "\Data2014VHLSS\\"

# Create list to save dataframes in
dataframes = []

# Select correct columns and datasets
datasets_info = {
    "Ho4.xlsx": [
        {"column": "m5a2ct", "new_name": "Expenditure_food"},
        {"column": "m5b2ct", "new_name": "Expenditure_non_food"}
    ],
    "muc1a.xlsx": [
        {"column": "m1ama", "new_name": "Household_member_id"}
    ],
    "muc7.xlsx": [
        {"column": "m7c27", "new_name": "Expenditure_housing"}
    ]
}

for file_name, col_info_list in datasets_info.items():
    file_path = f"{path_}\\{file_name}"

    for col_info in col_info_list:
        df = pd.read_excel(file_path, usecols=["diaban", "huyen", "tinh", "xa", "hoso", col_info["column"]])

        # Filter on Mekong Delta
        mekong_delta_provinces = ['TØnh Long An','TØnh TiÒn Giang', 'TØnh BÕn Tre', 'TØnh Trµ Vinh', 'TØnh VÜnh Long',
            'TØnh §ång Th¸p', 'TØnh An Giang', 'TØnh Kiªn Giang', 'TØnh CÇn Th¬', 'TØnh HËu Giang',
            'TØnh Sãc Tr¨ng', 'TØnh B¹c Liªu', 'TØnh Cµ Mau',
            80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96]
        df = df[df['tinh'].isin(mekong_delta_provinces)]

        provinces = {'TØnh Long An':"80", 'TØnh TiÒn Giang':"82", 'TØnh BÕn Tre':"83", 'TØnh Trµ Vinh':"84", 
                     'TØnh VÜnh Long':"86", 'TØnh §ång Th¸p':"87", 'TØnh An Giang':"89", 'TØnh Kiªn Giang':"91", 
                     'TØnh CÇn Th¬':"92", 'TØnh HËu Giang':"93", 'TØnh Sãc Tr¨ng':"94", 'TØnh B¹c Liªu':"95", 
                     'TØnh Cµ Mau':"96"}

        # Not each province uses the same name type, therefore we need to transform it
        def convert_province(val):
            if isinstance(val, str):
                return provinces.get(val.strip(), None)
            return val

        df['tinh'] = df['tinh'].apply(convert_province)

        # Rename columns
        df = df.rename(columns={
            "tinh": "Province",
            "huyen": "District",
            "xa": "Commune",
            "hoso": "Household",
            col_info["column"]: col_info["new_name"]
        })

        # Create Household ID
        df["Household_id"] = df["Province"].astype(str) + " - " + df['District'].astype(str) + " - " + df["Commune"].astype(str) + " - " + df["diaban"].astype(str) + " - " + df["Household"].astype(str)

        # Drop columns which we are not going to use
        df = df.drop(["Commune", "Household", "District", "Province", "diaban"], axis=1)

        # Delete Nan values and set them to zero
        df.fillna(0, inplace=True)

        # Groupby household id, everything can be grouped as a sum, except for household member id. We need to know household size, so the highest member id is used. 
        if file_name == "muc1a.xlsx":
            df = df.groupby("Household_id", as_index=False).max()
        else:
            df = df.groupby("Household_id", as_index=False).sum()

        dataframes.append(df)


In [24]:
# Combine all dataframes
df_expenditure = dataframes[0]
for df in dataframes[1:]:
    df_expenditure = df_expenditure.merge(df, on=["Household_id"], how="outer")

# fill in nan values with 0
df_expenditure = df_expenditure.fillna(0, inplace=False)
df_expenditure

,Household_id,Expenditure_food,Expenditure_non_food,Household_member_id,Expenditure_housing
0,80 - 794 - 27685 - 1 - 1,1520.0,2466,2,5340
1,80 - 794 - 27685 - 1 - 10,2658.0,3565,3,2600
2,80 - 794 - 27685 - 1 - 102,2992.0,5080,4,2150
3,80 - 794 - 27685 - 1 - 103,880.0,2420,1,5800
4,80 - 794 - 27685 - 1 - 105,2237.0,3950,5,2400
...,...,...,...,...,...
13792,96 - 973 - 32248 - 2 - 4,4938.0,14100,6,13600
13793,96 - 973 - 32248 - 2 - 5,5320.0,14690,10,3500
13794,96 - 973 - 32248 - 2 - 6,3506.0,10000,6,3000
13795,96 - 973 - 32248 - 2 - 7,4000.0,8000,4,3000


In [25]:
# Calculate total expenditure
df_expenditure['Yearly_expenditure_per_member'] = ((df_expenditure['Expenditure_food']*12) +  df_expenditure['Expenditure_housing'] +  df_expenditure['Expenditure_non_food']) /  df_expenditure['Household_member_id']

# Look at the quantiles. later, a child will have an expenditure between 0.25-0.5, and an adult betwen 0.5 and 0.75
df_expenditure = df_expenditure["Yearly_expenditure_per_member"].quantile([0.25, 0.5, 0.75])

# Set the quantiles in a dataframe
df_expenditure = pd.DataFrame({
    "Percentile": ["25%", "50%", "75%"],
    "Yearly_expenditure_per_member": df_expenditure.values
})

# Save to excel
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    df_expenditure.to_excel(writer, sheet_name="Expenditure_per_hh_member", index=False)

### Association

In [26]:
# Define path
path = os.getcwd()

path_ = path + "\Data2014VHLSS\\muc1a.xlsx"
association_data=pd.read_excel(path_, usecols=('diaban', 'tinh', 'xa', 'huyen','hoso', 'm1ac15a', 'm1ac15b', 'm1ac15c', 'm1ac15d'))

mekong_delta_provinces = ['TØnh Long An','TØnh TiÒn Giang', 'TØnh BÕn Tre' , 'TØnh Trµ Vinh', 'TØnh VÜnh Long', 'TØnh §ång Th¸p', 'TØnh An Giang', 'TØnh Kiªn Giang', 'TØnh CÇn Th¬', 'TØnh HËu Giang', 'TØnh Sãc Tr¨ng', 'TØnh B¹c Liªu', 'TØnh Cµ Mau',
80,81,82,83,84,85,86,87,88,89,80,81,92,93,94,95,96]
association_data = association_data[association_data['tinh'].isin(mekong_delta_provinces)]

provinces = {'TØnh Long An':"80",'TØnh TiÒn Giang':"82", 'TØnh BÕn Tre':"83" , 'TØnh Trµ Vinh':"84", 'TØnh VÜnh Long':"86", 'TØnh §ång Th¸p':"87", 'TØnh An Giang':"89", 'TØnh Kiªn Giang':"91", 'TØnh CÇn Th¬':"92", 'TØnh HËu Giang':"93", 'TØnh Sãc Tr¨ng':"94", 'TØnh B¹c Liªu':"95", 'TØnh Cµ Mau':96}

def convert_province(val):
    if isinstance(val, str):
        return provinces.get(val.strip(), None)
    return val 
association_data['tinh'] = association_data['tinh'].apply(convert_province)

# Rename columns
association_data = association_data.rename(columns = {'tinh':'Province', 'xa':'Commune', 'huyen':"District", 'hoso':'Household','m1ac15a':"Farmers_association", 'm1ac15b':"Womens_association", 'm1ac15c':"The_party", 'm1ac15d':"War_veterans_association"})

# Create new column to make it easier to match with other datasets
association_data["Household_id"] = association_data["Province"].astype(str) + " - " + association_data['District'].astype(str)+ " - " + association_data["Commune"].astype(str) + " - " + association_data["diaban"].astype(str) + " - " + association_data["Household"].astype(str)


# since in the data 1=yes, 2=no, change 2 to 0 to have a dummy variable
for i in ["Farmers_association", "Womens_association", "The_party", "War_veterans_association"]:
    association_data[i] = association_data[i].replace("Yes", 1)
    association_data[i] = association_data[i].replace("No", 0)

# Select relevant columns
association_data = association_data.drop(['diaban', 'Commune', 'Household', 'District', 'Province'], axis=1)

In [27]:
association_data.fillna(0, inplace=True)

# Group by household_id
associations = association_data.groupby(["Household_id"]).sum(min_count=1).reset_index()

associations_farmers = associations.merge(employ_data, on = "Household_id")
associations_farmers = associations_farmers.fillna(0)
associations_farmers = associations_farmers[associations_farmers['Farmers_association'].isin([0,1])]
associations_farmers = associations_farmers[associations_farmers['Worked_self_agri']>0]

data = associations_farmers['Farmers_association'].value_counts(normalize=True).reset_index()
data.columns = ['Farmers_association', 'Percentage']

# Change 0 to "no" and 1 to "yes"
data['Farmers_association'] = data['Farmers_association'].map({0: 'no', 1: 'yes'})

data = data.rename(columns= {"Farmers_association":"Association"})

with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    data.to_excel(writer, sheet_name="Member_association", index=False)

## Pop Housing Census 2019
From this dataset, the following is used:
- Occupations, employment types, and sectors
- Do people have experience and equipment
- Statistics about birth rate, death age and mean hh size
- Housing quality


In [28]:
# Define path
path = os.getcwd()

# Import data
path_sav = path + "\Data\\Sample Census 2019 (Q01-Q41).sav"
df=pd.read_spss(path_sav)

# Only select Mekong Delta
df_mekong = df[(df['MATINH'] >= 82) & ((df['MATINH']) <= 96)]
# In this survey, the district column is a float instead of int. we need to transform that first
df_mekong["MAHUYEN"] = df_mekong["MAHUYEN"].astype(int)
df_mekong = df_mekong[df_mekong['MAHUYEN']== district]
df_mekong["Household_id"] = df_mekong["MATINH"].astype(str) + " - " + df_mekong["MAHUYEN"].astype(str) + " - " + df_mekong["MAXA"].astype(str) + " - " + df_mekong["MADIABAN"].astype(str) + " - " + df_mekong["HOSO"].astype(str)
# Select rural area
df_mekong = df_mekong[df_mekong['TTNT'] == 2]

In [29]:
df_mekong

,MATINH,MAHUYEN,MAXA,MADIABAN,TTNT,Loaiphieu,HOSO,IDTV,Sokhau,C2,...,C38T,C38N,C39A,C39B,C40A,C40B,C41,C41K,Call_weight_FINAL,Household_id
7327369,89.0,894,30692.0,6.0,2.0,1.0,103.0,6.0,9.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,11.213699,89.0 - 894 - 30692.0 - 6.0 - 103.0
7327370,89.0,894,30692.0,6.0,2.0,1.0,103.0,7.0,9.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,11.213699,89.0 - 894 - 30692.0 - 6.0 - 103.0
7327371,89.0,894,30692.0,6.0,2.0,1.0,103.0,8.0,9.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,11.213699,89.0 - 894 - 30692.0 - 6.0 - 103.0
7327372,89.0,894,30692.0,6.0,2.0,1.0,103.0,9.0,9.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,10.621154,89.0 - 894 - 30692.0 - 6.0 - 103.0
7327373,89.0,894,30692.0,6.0,2.0,1.0,111.0,1.0,5.0,1.0,...,1.0,1995.0,0.0,1.0,NaN,NaN,NaN,,11.399624,89.0 - 894 - 30692.0 - 6.0 - 111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7357513,89.0,894,30724.0,17.0,2.0,1.0,3.0,4.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,13.330546,89.0 - 894 - 30724.0 - 17.0 - 3.0
7357522,89.0,894,30724.0,17.0,2.0,1.0,6.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,13.732358,89.0 - 894 - 30724.0 - 17.0 - 6.0
7357523,89.0,894,30724.0,17.0,2.0,1.0,6.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,12.753830,89.0 - 894 - 30724.0 - 17.0 - 6.0
7357524,89.0,894,30724.0,17.0,2.0,1.0,10.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,13.222895,89.0 - 894 - 30724.0 - 17.0 - 10.0


### Occupation distribution
Since the sectors of 2009 are generalized, the occupation distribution of 2019 is used. 

In [30]:
df_occupations = df_mekong[['Household_id', 'C27C', 'C26C', 'C30']]
# Use weight since this is on individual level
df_occupations = df_occupations.loc[df_occupations.index.repeat(df_mekong['Call_weight_FINAL'])].reset_index(drop=True)

# Translate the employment types to english instead of Vietnamese
transformation_work_type = {
    3: "Family_worker",
    1: "Other",
    2: "Other",
    4: "Employee",
    5: "Employee"}

df_occupations['C30'] = df_occupations['C30'].replace(transformation_work_type)
df_occupations = df_occupations.dropna()

# Define the occupations people have
df_occupations['C26C'] = pd.to_numeric(df_occupations['C26C'], errors='coerce')
conditions_occupation_C23 = [
    ((df_occupations['C26C'].between(6310, 6340)) | (df_occupations['C26C'].between(9201,9206))),
    (df_occupations['C26C'].between(6111, 6225)),
    (df_occupations['C26C'].between(9111, 9129)) | (df_occupations['C26C'].between(9311, 9629)),
    (df_occupations['C26C'].between(7111, 7422)) | (df_occupations['C26C'].between(7511, 7516)),
    (df_occupations['C26C'].between(1118, 5409))
]

# These are the labels related to the occupations above
choices = [
    "low_skilled_agri_worker",
    "skilled_agri_worker",
    "low_skilled_nonAgri",
    "manual_worker",
    "skilled_service_worker"
]


# Combine the occupations with the labels
df_occupations['C26C'] = np.select(conditions_occupation_C23, choices, default="other")

df_sectors = df_occupations
# It is now time to look at the sectors 
conditions_sectors = [
    df_occupations['C27C'].between(112,119), 
    df_occupations['C27C'].between(121, 129),
    df_occupations['C27C'] == 111, # ASSUMPTION THIS IS RICE
    df_occupations['C27C'].isin([310, 321, 322]),
    df_occupations['C27C'].between(510, 9900)
]

choices = [
    'Annual crops',
    'Perennial crops',
    'Rice',
    'Aquaculture',
    'Non_agri'
]
# Apply the new labels to the dataset.
df_occupations['C27C'] = np.select(conditions_sectors, choices, default='Other')

# Delete foresty etc for the scope
df_occupations = df_occupations[df_sectors['C27C'] != 'Other']

df_occupations = df_occupations[df_occupations['C27C'] != 9.0]
# Rename columns
df_occupations = df_occupations.rename(columns = {"C27C":"Sector", "C26C":"Occupation", "C30":"Employment_type"})

# Look at the distribution of sectors
df_sectors = df_occupations['Sector'].value_counts(normalize=True).reset_index()
df_sectors.columns = ['Sector', 'Probability']
df_sectors

# Save to the excel file the distribution of sectors
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    df_sectors.to_excel(writer, sheet_name="Sector_distribution", index=False)


In [31]:
# Occupation distribution
occupation_pct = (
    df_occupations.groupby('Sector')['Occupation']
      .value_counts(normalize=True)
      .mul(100)
      .rename('perc_occupation')
      .reset_index())

# Employment distribution
employment_pct = (
    df_occupations.groupby(['Sector', 'Occupation'])['Employment_type']
      .value_counts(normalize=True)
      .unstack(fill_value=0) 
      .mul(100) 
      .round(1)
      .reset_index()
)

# Delete the rows where all three columns are zero
employment_pct = employment_pct[
    ~( (employment_pct['Employee'] == 0) &
       (employment_pct['Family_worker'] == 0) &
       (employment_pct['Other'] == 0) )
]

# Drop the 9 column, which is created since employment type was in a few cases 9.0 instead of a normal value
# employment_pct = employment_pct.drop([9.0], axis = 1)
total_df = employment_pct.merge(occupation_pct, on = ['Sector', 'Occupation'])
total_df

with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    total_df.to_excel(writer, sheet_name="Occupation_and_employment", index=False)

### Experience

In [32]:
# Select relevant columns
knowledge_df = df_mekong[['Household_id','C28', 'C29A', "C29B", "C30", "C27C", "C26C"]]
# Use weight to make it more representative
df_replicated = knowledge_df.loc[knowledge_df.index.repeat(df_mekong['Call_weight_FINAL'])].reset_index(drop=True)

# Fill nan values with zero
df_replicated.fillna(0, inplace=True)

# Change 2 to zero since that means no
df_replicated["C28"] = df_replicated['C28'].replace(2, 0)
df_replicated["C29A"] = df_replicated['C29A'].replace(2, 0)
df_replicated["C29B"] = df_replicated['C29B'].replace(2, 0)
df_replicated.fillna(0, inplace=True)

# Rename columns
df_replicated = df_replicated.rename(columns = {"C28": "Training", "C29A":"3+Experience", "C29B":"Machines_equip", "C30":"Employment_type", "C27C":"Sector", "C26C":"Occupation"})

# Transform occupation codes
df_replicated['Occupation'] = pd.to_numeric(df_replicated['Occupation'], errors='coerce')
conditions_occupation_C23 = [
    ((df_replicated['Occupation'].between(6310, 6340)) | (df_replicated['Occupation'].between(9201,9206))),
    (df_replicated['Occupation'].between(6111, 6225)),
    (df_replicated['Occupation'].between(9111, 9129)) | (df_replicated['Occupation'].between(9311, 9629)),
    (df_replicated['Occupation'].between(7111, 7422)) | (df_replicated['Occupation'].between(7511, 8350)),
    (df_replicated['Occupation'].between(1118, 5409))
]

choices_occupation = [
    "low_skilled_agri_worker",
    "skilled_agri_worker",
    "low_skilled_nonAgri",
    "manual_worker",
    "skilled_service_worker"
]
# Change occupation codes to the strings they represent
df_replicated['Occupation'] = np.select(conditions_occupation_C23, choices_occupation, default="other")

# Set sectors
conditions_sector = [
    df_replicated['Sector'].between(111, 119),
    df_replicated['Sector'].between(121,129),
    df_replicated['Sector'].between(131, 240),
    df_replicated['Sector'].between(311, 322),
    df_replicated['Sector'].between(510,9900),
]

choices_sectors = [
    'Annual crops',
    'Perennial crops',
    'Rice',
    'Aquaculture',
    'Non_agri'
]

df_replicated['Sector'] = np.select(conditions_sector, choices_sectors, default='No_sector')

# Change employment codes to strings they represent
employments = {1: "owner",2: "self_employed",3: "family_labour",4: "cooperative",5: "salary_earner",0: "Not_employed"}
df_replicated['Employment_type'] = df_replicated['Employment_type'].map(employments)

percentages_knowledge = (
    df_replicated.groupby('Occupation')[['Training', '3+Experience', 'Machines_equip']]
    .mean()
    .mul(100)      
    .round(1)      
)

# Save to excel
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    percentages_knowledge.to_excel(writer, sheet_name="Experience_per_occupation", index=True)


### Birth rate

In [33]:
# Select births, household_id and household size
df_birth_rate = df_mekong[['C40A', "C40B", "Household_id", "Sokhau"]]

df_birth_rate.fillna(0, inplace=True)
df_birth_rate = df_birth_rate.groupby("Household_id", as_index=False).agg({'C40A': 'sum', 'C40B': 'sum', 'Sokhau': 'mean'})

# Information on household size
df_birth_rate['Sokhau'].mean()
df_birth_rate['Sokhau'].std()

# Birth rate
Number_of_childs = df_birth_rate["C40A"].sum() + df_birth_rate['C40B'].sum() # Take the boys and girls together
total_population = df_birth_rate['Sokhau'].sum()
birth_rate = Number_of_childs / total_population

### Death rate
We need to use the second part of the dataset for the death rate

In [34]:
# Define path
path = os.getcwd()

# Import data
path_sav2 = path + "\Sample Census 2019 (Q42-Q49).sav"
df2=pd.read_spss(path_sav2)

# Only select Mekong Delta
df_mekong2 = df2[(df2['MATINH'] >= 82) & ((df2['MATINH']) <= 96)]

# Set district type to int instead of float, and select the correct district
df_mekong2["MAHUYEN"] = df_mekong2["MAHUYEN"].astype(int)
df_mekong2 = df_mekong2[df_mekong2['MAHUYEN']== district]

# Create household id
df_mekong2["Household_id"] = df_mekong2["MATINH"].astype(str) + " - " + df_mekong2["MAHUYEN"].astype(str) + " - " + df_mekong2["MAXA"].astype(str) + " - " + df_mekong2["MADIABAN"].astype(str) + " - " + df_mekong2["HOSO"].astype(str)
# Select rural area
df_mekong2 = df_mekong2[df_mekong2['TTNT'] == 2]

# Use weight
df_mekong2 = df_mekong2.loc[df_mekong2.index.repeat(df_mekong2['Call_weight_HH'])].reset_index(drop=True)


#### Combine birth rate and death rate in 1 dataframe

In [35]:
# Combine all statistics in 1 dataframe
statistics_df = pd.DataFrame({
    'Statistic': ['Mean_death_age', 'Std_dev_death_age', 'Birth_rate', 'Mean_hh_size','Std_dev_hh_size' ],
    'Value': [df_mekong2['C47'].mean(), df_mekong2['C47'].std(), birth_rate, df_birth_rate['Sokhau'].mean(),df_birth_rate['Sokhau'].std()]
})

# Save to excel
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    statistics_df.to_excel(writer, sheet_name="Statistics_population_size", index=False)


### House Quality
we need the third part of the dataset for this

In [36]:
# Define path
path = os.getcwd()

# Import data
path_sav3 = path + "\Sample Census 2019 (Q50-Q65).sav"
df3=pd.read_spss(path_sav3)

# Only select Mekong Delta
df_mekong3 = df3[(df3['MATINH'] >= 82) & ((df3['MATINH']) <= 96)]
# Set district type to int instead of float, and select the correct district
df_mekong3["MAHUYEN"] = df_mekong3["MAHUYEN"].astype(int)
df_mekong3 = df_mekong3[df_mekong3['MAHUYEN']== district]
df_mekong3["Household_id"] = df_mekong3["MATINH"].astype(str) + " - " + df_mekong3["MAHUYEN"].astype(str) + " - " + df_mekong3["MAXA"].astype(str) + " - " + df_mekong3["MADIABAN"].astype(str) + " - " + df_mekong3["HOSO"].astype(str)
# Select rural area
df_mekong3 = df_mekong3[df_mekong3['TTNT'] == 2]
df_mekong3 = df_mekong3.loc[df_mekong3.index.repeat(df_mekong3['Call_weight_HH'])].reset_index(drop=True)

In [37]:
# Select relevant columns
df_housing = df_mekong3[['Household_id', "C55", "C56", "C57", "C59"]]

# Rename columns
df_housing = df_housing.rename(columns = {"C55":"Main_construction", "C56":"Roof", "C57":"Outer_walls", "C59":"Ownership"})

# Replace values. 2 means that the distruction is not stable
df_housing['Main_construction'] = df_housing['Main_construction'].replace(2,0)
df_housing['Roof'] = df_housing['Roof'].replace(2,0)
df_housing['Outer_walls'] = df_housing['Outer_walls'].replace(2,0)

# Calculate average housing quality based on the three factors
df_housing['Average_housing_quality'] = df_housing[['Main_construction', 'Roof', "Outer_walls"]].mean(axis=1)

# Place averything in a dataframe
housing_quality_stats = pd.DataFrame({
    'hh_quality': ['mean', 'std_dev'],
    'value': [df_housing['Average_housing_quality'].mean(), df_housing['Average_housing_quality'].std()]
})

# Save to excel
with pd.ExcelWriter(f"model_input_data_{district}.xlsx", mode='a', engine='openpyxl') as writer:
    housing_quality_stats.to_excel(writer, sheet_name="Housing_quality", index=False)
housing_quality_stats

,hh_quality,value
0,mean,0.791718
1,std_dev,0.357903
